Copyright (c) 2023, Douglas Santry
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, is permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this
   list of conditions and the following disclaimer.
2. Redistributions in binary form must reproduce the above copyright notice,
   this list of conditions and the following disclaimer in the documentation
   and/or other materials provided with the distribution.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND
ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR
ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES
(INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES;
LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND
ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
(INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS
SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.


In [1]:
import numpy as np

import matplotlib.pyplot as plt
import string
import tensorflow as tf
from keras.utils import np_utils
from math import ceil as ceil

from tensorflow.keras import layers
from tensorflow.keras import losses
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures, AutoConfig

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [2]:
from datasets import load_dataset
from transformers import DataCollatorWithPadding

In [3]:
DataLocation = "/Users/dsantry/Scratch/Data/BBC/bbc/"

In [4]:
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=5)

tokenizer = AutoTokenizer.from_pretrained ("bert-base-uncased")


2023-11-09 19:55:14.906103: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-09 19:55:14.906175: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
model.compile (optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

In [6]:
model.summary ()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  3845      
                                                                 
Total params: 109,486,085
Trainable params: 109,486,085
Non-trainable params: 0
_________________________________________________________________


In [7]:
%run Project_9_3_data.py

In [8]:
type (W)

datasets.arrow_dataset.Dataset

In [9]:
Z = DatasetDict()

Z["train"] = W

In [10]:
def tokenize_function(example):
    return tokenizer(example["Text"], truncation=True)

In [11]:
tokenizedDS = Z.map (tokenize_function, batched=True)

Map:   0%|          | 0/2676 [00:00<?, ? examples/s]

In [12]:
type (tokenizedDS)

datasets.dataset_dict.DatasetDict

In [13]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

K = {k: v for k, v in K.items() if k not in ["DocID", "Chunk"]}
[len(x) for x in K["input_ids"]]

batch = data_collator(K)
{k: v.shape for k, v in batch.items()}

In [14]:
X = tokenizedDS["train"].to_tf_dataset (
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["Label"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size = 8
)

/Users/dsantry/miniforge3/lib/python3.9/site-packages/datasets/arrow_dataset.py:388: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [15]:
type (X)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [16]:
h = model.fit (X, epochs=2)

Epoch 1/2


2023-11-09 19:55:18.117574: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-11-09 19:55:27.398915: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


335/335 [==============================] - 1732s 5s/step - loss: 0.2808 - accuracy: 0.9256
Epoch 2/2
335/335 [==============================] - 1966s 6s/step - loss: 0.0707 - accuracy: 0.9854


In [17]:
Verify = tokenizedDS["train"].to_tf_dataset (
    columns=["attention_mask", "input_ids", "token_type_ids"],
    collate_fn=data_collator,
    batch_size = 8
)

In [18]:
K = model.predict (Verify)

K

2023-11-09 20:56:58.255912: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


335/335 [==============================] - 731s 2s/step


TFSequenceClassifierOutput(loss=None, logits=array([[ 5.2651978 , -0.49600106, -1.4951605 , -1.5954058 , -1.2668053 ],
       [ 5.3377504 , -0.5181489 , -1.5402131 , -1.6471913 , -1.1874607 ],
       [ 5.3956923 , -0.5316864 , -1.5965341 , -1.7333562 , -1.067245  ],
       ...,
       [-2.0535228 , -0.6131928 , -1.7345933 , -1.6756494 ,  5.4605665 ],
       [-1.9186448 , -0.88515013, -1.8667494 , -1.5000626 ,  5.4625616 ],
       [-1.7839164 , -0.9852357 , -1.8003498 , -1.5718783 ,  5.541689  ]],
      dtype=float32), hidden_states=None, attentions=None)

In [20]:
guesses = K.logits.argmax (1)

In [21]:
sum (Z["train"]["Label"] == guesses) / len (guesses)

0.9943946188340808

In [ ]:
K = Z["train"]


In [ ]:
prediction = guesses == K["label"]

In [ ]:
prediction